In [1]:
import sys
sys.path.append('../../')

from bokehComponents.Dashboard import Dashboard
from bokehComponents.BokehComponents import BokehControl,BufferedQueryInterface,QueryTableComponent

from bokeh.io import output_notebook,output_file
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models.widgets import Panel, Tabs #, DataTable, DateFormatter, TableColumn, Tabs, 
from bokeh.models import ColumnDataSource
from bokeh.layouts import layout
from bokeh.plotting import figure,show,output_file
from bokeh.models import Div

from datetime import datetime,timedelta
import pandas_datareader as pdr


In [2]:
from bokeh.plotting import figure, output_file, show



# 5 The Data Grid
Frequently, a data grid is required as part of a dashboard. It may be the result of a query, or a general data dump. One of the main classes I created in bokehComponents was the data grid. It handles all of the nasty events related to selecting indexes, updating data sources, and working with graphs.


### Status: In Development
Note, the library is in active development, and virtually not method name or class name, or parameter is stable.

## 5.1 Creating Data using the BufferedQueryInterface
As is usual, we begin our work by defining a data source. We are merely repeating the previous data source we have constructed that hasd a well behaved append and delete action defined.

In [3]:

class ExampleComplexQuery(BufferedQueryInterface):

    def appendNextAction(self,indicesIn= None):
        #
        ## First, you are given selected data indices. Sometimes this may be null. in this action we will ignore this index.

        assert indicesIn or not indiciesIn # do something with indices if you want

        #
        ## Second, do something with self.data

        next_day = self.current_date+ timedelta(days=3)
        data=pdr.get_data_yahoo('AAPL',self.current_date,next_day)
        self.data['open_list'].append(data['Open'][0])
        self.data['close_list'].append(data['Close'][0])
        self.data['date_time'].append(next_day)
        assert len(self.data['open_list']) == len(self.data['close_list'])
        assert len(self.data['close_list']) == len(self.data['date_time'])

        self.current_date = next_day 

    def deleteAction(self,indicesIn= None):
        #
        ## Remove all the indices, from largest to smallest
        print("Deleting",indicesIn)
        tables = ['open_list','close_list','date_time']
        indicesIn.sort(reverse=True)
        for i in indicesIn:
            for tableName in tables:
                self.data[tableName].pop(i)

    def load_data_buffer(self):
        try:
            if (self.init):
                return
        except:
            pass
        self.init = 1
        start_init=self._settings['start_date']
        end_init=self._settings['end_date']     
        self.current_date = end_init 
        # Grab data
        start=start_init
        end=end_init 

        data=pdr.get_data_yahoo('AAPL',start,end)
        data_sorted=data.sort_index(axis=0,ascending=True)
        date_list=list(data_sorted.index)

        # Save the data elements
        open_list=list(data_sorted['Open'])
        close_list=list(data_sorted['Close'])
        date_time=[datetime.strptime(str(d),'%Y-%m-%d %H:%M:%S').date() for d in date_list]

        #
        ## First, create the data
        self.data = {
            'open_list':open_list,
            'close_list':close_list,
            'date_time':date_time,
            }

        #
        ## Second, register the append action so it can be used.
        self.registerAction('append',ExampleComplexQuery.appendNextAction)
        self.registerAction('delete',ExampleComplexQuery.deleteAction)
        self.registerAction('kill',ExampleComplexQuery.deleteAction)

### 5.1 Invoke the defauly QueryTable
It is possible use the graphic timeseries out of the box

In [4]:
class GraphDashboard(Dashboard):
    def setTableClass(self,tableClass):
        self.tableClass = tableClass
        
    def getLayout(self):
        settings = {'buffered_query_interface':ExampleComplexQuery({'start_date':datetime(2017,4,26),
                                               'end_date':datetime(2017,5,26)}),
                   'id_field':'open_list',
                   'width':600,
                   'height':300}

        self.t = self.tableClass(settings)
        l = layout([self.t.getBokehComponent()],sizing_mode='fixed')         
        return l    

In [5]:
# Important -- each cell you run will start and override the event handler for the whole notebook
d = GraphDashboard()
d.setTableClass(QueryTableComponent)
d.showDashboard()

Loading BokehJS ...

### 5.2 Create a custom handle for the select method

In [15]:
# Important -- each cell you run will start and override the event handler for the whole notebook
class AlertSelectTable(QueryTableComponent):
    def do_handle_select(self,attr,old,new):
        dat = self._settings['buffered_query_interface'].data        
        print(attr)
        print(old)
        print(new)
        for index_key in new:
            print ("row selected in data query: ", end=',')
            for col_key in dat:
                print (col_key,"-",dat[col_key][index_key], end=',')
            print ("")
            
d = GraphDashboard()
d.setTableClass(AlertSelectTable)
d.showDashboard()

Loading BokehJS ...

### 5.3 Create a delete handle for the select method

In [16]:
# Important -- each cell you run will start and override the event handler for the whole notebook
class RemoveSelectTable(QueryTableComponent):
    def do_handle_select(self,attr,old,new):
        print('pre delete',str(new))
        self.removeSelected()
        print('post delete',str(new))
d = GraphDashboard()
d.setTableClass(RemoveSelectTable)
d.showDashboard()

Loading BokehJS ...

### 5.4 It is possible to inspect all data
These member variables may change. At present it is possible to inspect all data 

In [17]:


display(d)
display(d.t)
display(d.t._settings)
display(d.t._settings['buffered_query_interface'])
display(d.t._settings['buffered_query_interface'].data)

{'buffered_query_interface': <__main__.ExampleComplexQuery at 0x7fa11b0f7470>,
 'id_field': 'open_list',
 'width': 600,
 'height': 300}

{'open_list': [144.47000122070312,
  143.9199981689453,
  144.08999633789062,
  145.10000610351562,
  147.5399932861328,
  145.58999633789062,
  146.52000427246094,
  146.75999450683594,
  149.02999877929688,
  153.8699951171875,
  153.6300048828125,
  152.4499969482422,
  154.6999969482422,
  156.00999450683594,
  155.94000244140625,
  153.60000610351562,
  151.27000427246094,
  153.3800048828125,
  154.0,
  154.89999389648438,
  153.83999633789062,
  153.72999572753906,
  154.0],
 'close_list': [143.67999267578125,
  143.7899932861328,
  143.64999389648438,
  146.5800018310547,
  147.50999450683594,
  147.05999755859375,
  146.52999877929688,
  148.9600067138672,
  153.00999450683594,
  153.99000549316406,
  153.25999450683594,
  153.9499969482422,
  156.10000610351562,
  155.6999969482422,
  155.47000122070312,
  150.25,
  152.5399932861328,
  153.05999755859375,
  153.99000549316406,
  153.8000030517578,
  153.33999633789062,
  153.8699951171875,
  153.61000061035156],
 'date_time'